# **Lyrics Generation using RNN**
### (can be generalised as coherent / meaningful text generation)

#### **Importing TensorFlow and other libraries**

In [0]:
import tensorflow as tf

import numpy as np
import os
import time

#### **Specifying the path of the text file to process**

In [0]:
path_to_file = 'lyrics_dataset.txt'

#### **Opening the text file in read mode**

In [0]:
# Opening the text file in read mode and standard encoding it
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

In [0]:
# A look at the first 250 characters in text
print(text[:100])

In [0]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

#### **Preprocessing of text i.e from strings to numerical representation**
##### Creating lookup-tables for char->num and num->char

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
print('{ ===========>')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n==========>}')

In [0]:
# Show how the first 20 characters from the text are mapped to integers
print ('{} ==> characters mapped to int ==> {}'.format(repr(text[:20]), text_as_int[:20]))

#### **Creating training examples and targets**
Broke the text into chunks of seq_length + 1 , if text is => "Alright" 

Then input sequence becomes => "Alrigh"

The output sequence becomes => "lright"

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()] , end = "")

In [0]:
# Using batch method converted individual characters to sequences of desired size
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

#### **Mapping function**

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

#### **Creating training batches for splitting text into manageable sequences**

In [0]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units 
rnn_units = 1500 # keep between (1024 -> 1800) for best results 

### **Buliding the Model**
#### **Four layers were used :**
######  **1. Embedding layer :** The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions
######  **2. GRU layer :** A type of RNN with size units=rnn_units (LSTM could also be used here.)
###### **3. Dense layer :** The output layer, with vocab_size outputs and 'RELU' as the activation fuction 
###### **4. Dropout layer :** Benifits regularisation and prevents overfitting  


In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
                               
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
  
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),

    tf.keras.layers.Dense(vocab_size,activation='relu'),
    
    tf.keras.layers.Dropout(0.2),
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

#### **Model Summary**

In [0]:
model.summary()

In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [0]:
sampled_indices

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

#### **Attaching an optimizer, and a loss function**
###### **tf.keras.losses.sparse_categorical_crossentropy** loss function works in this case because it is applied across the last dimension of the predictions.

In [130]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 98)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.5856647


#### **Adam optimiser gives the best result hands down**

In [0]:
model.compile(optimizer='adam', loss=loss)

#### **Configuring the checkpoints , ensuring that the checkpoints are saved during training**

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#### **Rounds of training -> EPOCHS**

In [0]:
# Configure it according to the loss you get at the end.
# Ensure that the loss is between 1.4 to 1.1 for best meaningful text generation ensuring that it is always new lyrics/text , NOT same as in the training set
# If the loss becomes less than 1 , then a lot of same text would be generated
# Obviously we don't want the same , but something new

EPOCHS=25 

In [0]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

#### **Generating text**

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

#### **Prediction Loop : the code block that generates the text**

In [0]:
def generate_text(model, chars_to_generate , temp , start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = chars_to_generate

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = temp

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

#### **Generation (finally :))**

In [0]:
from numpy import arange

# Number of characters to generate (keep between 250 to 500)
chars_to_generate = 500 

# Printing the generated text
# Temperature 1.0 gives the craziest output and 0.1 gives the lowest varience
# Keeping the temperature 0.35 gives best meaningful / coherent text.

# Give the seed string as the first word of generate text
print(generate_text(model , chars_to_generate , 0.35 , start_string=u"Love "))

# Uncomment below to check the variences ==>

# for i in arange(0.1,1.1,0.1):
#   print("==============")
#   print("FOR TEMP : {} ".format(i))
#   print("==============")
#   print(generate_text(model , chars_to_generate , i , start_string=u"Love "))
#   print()